In [2]:
import numpy as np
import pandas as pd
import tqdm
from sklearn.preprocessing import MultiLabelBinarizer

In [3]:
res = pd.read_pickle("../data/dataframes/output/manual_annotation_df.pkl")
res

,PT,AU,TI,SO,DE,AB,C1,EM,TC,PY,...,Society,Economy,Environment,EU,DST,ANN,ANN_DT,ANN_SDG,Relevance,ANN_DF
967427,J,"Cho, WJ; Kim, HJ; Jung, DH; Han, HJ; Cho, YY",Hybrid Signal-Processing Method Based on Neura...,SENSORS,ion-selective electrode; sensor array; two-poi...,"In closed hydroponics, fast and continuous mea...","[Cho, Woo-Jae; Kim, Hak-Jin; Jung, Dae-Hyun; H...",er8er@snu.ac.kr; kimhj69@snu.ac.kr; jeoguss@gm...,8,2019,...,True,False,False,False,True,DTTSDG,NaN,1,3,1
739958,J,"Bahbouh, K; Wagner, JR; Morency, C; Berdier, C",Travel demand corridors: Modelling approach an...,JOURNAL OF TRANSPORT GEOGRAPHY,Demand corridor; Origin and Destination (OD); ...,"In an ideal world, transportation networks and...","[Bahbouh, Kinan; Morency, Catherine] Ecole Pol...",Kinan.bahbouh@polymtl.ca,5,2017,...,True,False,False,True,True,DTTSDG,NaN,1,3,1
1073691,J,"Yilmaz, A; Ayan, K",Cancer risk analysis by fuzzy logic approach a...,TURKISH JOURNAL OF ELECTRICAL ENGINEERING AND ...,Fuzzy logic; artificial intelligence; cancer; ...,Cancer is the leading life-threatening disease...,"[Yilmaz, Atinc] Halic Univ, Dept Comp Engn, Is...",atincyilmaz@halic.edu.tr,11,2013,...,True,False,False,False,True,ASDGDT,NaN,1,3,1
862859,J,"Wei, LX; Zhou, BS; Ma, XC; Chen, DX; Zhang, JY...",Lightning: A High-Efficient Neighbor Discovery...,IEEE COMMUNICATIONS LETTERS,Low Duty Cycle WSNs; Neighbor Discovery; Deter...,"In wireless sensor networks (WSNs), due to the...","[Wei, Liangxiong; Zhou, Beisi; Ma, Xichu; Chen...",jianpeng@scu.edu.cn; chenliangyin@scu.edu.cn,24,2016,...,True,False,False,False,True,ODT,NaN,0,1,1
61611,C,"Gondara, L",Medical image denoising using convolutional de...,2016 IEEE 16TH INTERNATIONAL CONFERENCE ON DAT...,Image denoising; denoising autoencoder; convol...,Image denoising is an important pre-processing...,"[Gondara, Lovedeep] Simon Fraser Univ, Dept Co...",lgondara@sfu.ca,162,2016,...,True,False,False,False,True,DTTSDG,NaN,1,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
844349,C,"Capozzoli, A; Primiceri, G",Cooling systems in data centers: state of art ...,SUSTAINABILITY IN ENERGY AND BUILDINGS: PROCEE...,Data center; Cooling technologies,"The growing number, size, complexity and energ...","[Capozzoli, Alfonso; Primiceri, Giulio] Polite...",alfonso.capozzoli@polito.it,83,2015,...,True,False,False,True,True,DTA,NaN,0,2,1
419668,J,"Ouallouche, F; Ameur, S",Rainfall detection over northern Algeria by co...,APPLIED WATER SCIENCE,Rainfall detection; MSG; TRMM; ANN,"In this paper, a new method to delineate rain ...","[Ouallouche, Fethi; Ameur, Soltane] Mouloud Ma...",ouafethi_04@yahoo.fr; ameursoltane@yahoo.com,6,2016,...,False,False,True,False,True,IR,NaN,0,1,1
411808,C,"Kato, H; Yamauchi, K",Quick MPPT microconverter using a limited gene...,2015 INTERNATIONAL CONFERENCE ON SUSTAINABLE E...,limited general regression neural network (LGR...,The recent development of photovoltaic electri...,"[Kato, Hidekazu; Yamauchi, Koichiro] Chubu Uni...",tp14008-2929@sti.chubu.ac.jp; yamauchi@cs.chub...,5,2015,...,False,False,True,False,True,DTTSDG,NaN,0,3,1
863893,C,"Lv, C; Qiao, L",Evenly-Distributed Clustering for Wireless Sen...,2016 IEEE INTERNATIONAL CONFERENCE OF ONLINE A...,WSN; Routing Protocols; Energy-Efficient,Users can accurately monitor a remote field co...,"[Lv, Can; Qiao, Lin] Tsinghua Univ, Dept Comp ...",lcpk0411@sina.com; qiaolin@tsinghua.edu.cn,0,2016,...,True,False,False,False,True,DTA,NaN,0,2,1


In [14]:
res = res[res['Relevance'] == "5"]
test = res.AB.to_list()
a = iter(test)

In [24]:
next(a)

StopIteration: 

In [22]:
f = "filtered"
# f = "unfiltered"

In [4]:
if f == "filtered":
    res2 = res[res['PY>=2019'] > 0]
    res2 = res2[res2['total_publications'] > 1]
    # condition_citations = res2['total_citations'] > 1
    # res2 = res2[condition_citations]
elif f == "unfiltered":
    res2 = res

In [5]:
c_sdg = {f'SDG{i}':0 for i in range(1,18)}
c_dt = {'IOT': 0, 'AI': 0, 'computing_infrastructure': 0, 'additive_manufacturing': 0, 'big_data': 0,'robotics': 0, 'blockchain': 0}
c_sdg_dt = {f'SDG{i}':c_dt.copy() for i in range(1,18)}
emails = set()

idx = {name: ind for ind, name in enumerate(res2.columns, start=1)}
c = 0
for row in res2.itertuples(name=None, index=True):
    lsdg = []
    ldt = []
    lsdg_dt = []
    for sdg in c_sdg:
        if row[idx[sdg]] > 1:
            emails.add(row[0])
            # c_sdg[sdg] += 1
            lsdg.append(sdg)
            # Inter
            for dt in c_dt:
                if row[idx[dt]] > 1:
                    lsdg_dt.append((sdg, dt))
    for dt in c_dt:
        if row[idx[dt]] > 1:
            emails.add(row[0])
            ldt.append(dt)

    size_sdg, size_dt, size_sdg_dt = len(lsdg), len(ldt), len(lsdg_dt)
    for s in lsdg:
        c_sdg[s] += 1 / (size_sdg+size_dt)
    for s, d in lsdg_dt:
        c_sdg_dt[s][d] += 1 / size_sdg_dt
    for d in ldt:
        c_dt[d] += 1/(size_sdg+size_dt)



In [6]:
len(emails)

481354

In [7]:
df_sdg = pd.Series(c_sdg, name="total_sdg").round(1)
df_sdg

SDG1      4170.9
SDG2      2567.9
SDG3     17510.5
SDG4      5990.4
SDG5      1289.5
SDG6     16088.9
SDG7      5552.7
SDG8      2644.2
SDG9      1323.5
SDG10      894.1
SDG11    11292.0
SDG12    10309.5
SDG13    50380.2
SDG14    20807.9
SDG15     9859.7
SDG16     3284.7
SDG17     2494.8
Name: total_sdg, dtype: float64

In [8]:
df_dt = pd.Series(c_dt, name='total_dt').round(1)
df_dt

IOT                          39557.1
AI                          176222.3
computing_infrastructure     24405.6
additive_manufacturing       14714.3
big_data                     12757.0
robotics                     43512.1
blockchain                    3724.2
Name: total_dt, dtype: float64

In [9]:
df_sdg_dt = pd.DataFrame.from_dict(c_sdg_dt, orient='index').round(1)
df_sdg_dt

,IOT,AI,computing_infrastructure,additive_manufacturing,big_data,robotics,blockchain
SDG1,8.1,97.8,3.5,2.6,19.2,15.1,10.7
SDG2,20.3,129.4,7.2,2.6,12.7,39.4,3.9
SDG3,219.2,1306.0,71.7,88.1,174.7,220.0,62.6
SDG4,70.7,414.9,25.4,19.1,48.4,75.7,5.9
SDG5,1.3,20.4,1.9,0.6,3.2,1.2,0.0
SDG6,191.2,1044.3,67.0,77.6,31.2,123.3,3.7
SDG7,1446.7,1857.7,887.1,78.3,501.7,411.8,81.6
SDG8,48.3,149.1,41.5,48.9,37.4,29.7,21.2
SDG9,62.6,158.7,35.2,55.0,50.1,25.6,9.7
SDG10,18.9,81.7,12.1,4.9,14.6,18.3,2.7


In [10]:
a = pd.concat([df_sdg_dt, df_sdg.to_frame()], axis=1)
b = pd.concat([a, df_dt.to_frame().T])
with pd.ExcelWriter('../data/email_count.xlsx', mode='a') as writer:
    b.to_excel(writer, sheet_name="filtered_no_citations")

In [2]:
df = pd.read_pickle('../data/dataframes/all_nodups_shorten.pkl')
df

,AF,EM,PY,UT,WC,TC,lst_SDG,lst_DT
0,"Gusiatin, Zygmunt M.; Kulikowska, Dorota; Klik...",mariusz.gusiatin@uwm.edu.pl; dorotak@uwm.edu.p...,2020,WOS:000570328900001,"Environmental Sciences; Public, Environmental ...",14,{SDG12},{}
1,"Vitko, Tadeo G.; Suffet, Irwin H. (Mel)",tvitko@cox.net,2021,WOS:000635505300001,"Engineering, Environmental; Environmental Scie...",2,{SDG12},{}
2,"Ntasiou, M.; Andreou, E.",eandreou3@gmail.com,2017,WOS:000410925400094,Construction & Building Technology; Engineerin...,6,{SDG12},{}
3,"Wu, Lipeng; Zhang, Shirong; Chen, Mengmeng; Li...",xiaodongding2004@163.com,2021,WOS:000744257400019,Biotechnology & Applied Microbiology; Engineer...,1,{SDG12},{}
4,"Staicu, Daniela",Danielastaicu@gmail.com,2019,WOS:000501603000009,Business,1,"{SDG12, SDG13}",{}
...,...,...,...,...,...,...,...,...
2798083,"Dieffenderfer, James; Wilkins, Michael; Hood, ...",mdaniel6@ncsu.edu; aybozkur@ncsu.edu,2016,WOS:000399395700065,"Engineering, Electrical & Electronic; Remote S...",9,{},{IOT}
2798084,"Dinesh, Karthik; Xiong, Mulin; Adams, Jamie; D...",NaN,2016,WOS:000402630000005,"Engineering, Electrical & Electronic",1,{},{IOT}
2798085,"Ding, Caiwen; Heidari, Soroush; Wang, Yanzhi; ...",cading@syr.edu; soroush.heidari@okstate.edu; y...,2016,WOS:000390094700044,"Engineering, Electrical & Electronic",9,{},{IOT}
2798086,"Ding, Caiwen; Li, Hongjia; Hu, Jingtong; Liu, ...",cading@syr.edu; hli42@syr.edu; jthu@okstate.ed...,2016,WOS:000391829200039,"Computer Science, Hardware & Architecture; Eng...",1,{},{IOT}


In [105]:
df_test = df.sample(100000)
df_test

,AF,EM,PY,UT,WC,TC,lst_SDG,lst_DT
2788046,"Nguyen, Hoang D.; Jiang, Xinyi; Poo, Danny Chi...",hoangnguyen@comp.nus.edu.sg; xinyi@comp.nus.ed...,2015,WOS:000364813200008,"Computer Science, Information Systems; Compute...",6,{},{IOT}
308273,"Ma'mun, S.; Svendsen, Hallvard F.; Bendiyasa, ...",sholeh.mamun@uii.ac.id,2018,WOS:000457694200008,"Engineering, Mechanical",3,{SDG13},{}
1241448,"Watson, Laurel B.; Flores, Mirella J.; Grotewi...",watsonlb@umkc.edu,2018,WOS:000442400200003,"Psychology, Multidisciplinary; Women's Studies",4,{SDG5},{}
1728017,"Resalat, Seyed Navid; Saba, Valiallah",vsaba@aut.ac.ir,2015,WOS:000361628300002,"Engineering, Electrical & Electronic; Imaging ...",16,{},{AI}
1370847,"Wang, Shui-Hua; Govindaraj, Vishnu Varthanan; ...",shuihuawang@ieee.org; g.vishunuvarathanan@klu....,2021,WOS:000598377700003,"Computer Science, Artificial Intelligence; Com...",142,{},{AI}
...,...,...,...,...,...,...,...,...
1415134,"Chinchilla, Monica; Santos-Martin, David; Carp...",micarpin@ing.uc3m.es,2021,WOS:000591382100005,"Energy & Fuels; Engineering, Chemical",21,{},{AI}
523781,"Gaubert, Julie; Rodolfo-Metalpa, Riccardo; Gre...",julieg1907@gmail.com,2020,WOS:000512364900026,Biotechnology & Applied Microbiology,7,"{SDG13, SDG14}",{}
1282181,"Lawley, Christopher J. M.; Tschirhart, Victori...",christopher.lawley@canada.ca,2021,WOS:000686005900002,Geology; Mineralogy; Mining & Mineral Processing,3,{},{AI}
2047306,"Wang, Chien-Chih; Chang, Cheng-Ding; Jiang, Be...",ieccwang@mail.mcut.edu.tw,2012,WOS:000312421800106,"Computer Science, Artificial Intelligence; Eng...",1,{},{AI}


In [126]:
def formalize(dataframe) -> pd.DataFrame:

    # Lists of columns for SDG and DT
    lst_dt = ['AI',	'big_data',	'IOT',	'computing_infrastructure',	'blockchain',	'robotics',	'additive_manufacturing']
    lst_dt.sort(key=lambda v: v.lower())
    lst_sdg = list(f'SDG{i}' for i in range(1,18))

    # scikit learn module for transforming list values into Dummies columns
    # new DF with index and lst_SDG exploded into dummy columns
    mlb = MultiLabelBinarizer()
    df_sdg = pd.DataFrame(mlb.fit_transform(dataframe['lst_SDG']),columns=mlb.classes_, index=dataframe.index).astype(bool)
    # new DF with index and lst_DT exploded into dummy columns
    mlb = MultiLabelBinarizer()
    df_dt = pd.DataFrame(mlb.fit_transform(dataframe['lst_DT']),columns=mlb.classes_, index=dataframe.index).astype(bool)

    # Merging
    res = dataframe.merge(df_sdg, left_index=True, right_index=True).merge(df_dt, left_index=True, right_index=True)

    # SDG, DT, Intersection,  Env, Eco, Soc columns
    cond_soc = res['SDG1'] | res['SDG2'] | res['SDG3'] | res['SDG4'] | res['SDG5'] | res['SDG6'] | res['SDG7'] | res['SDG11'] | res['SDG16']
    res.loc[:, 'Society'] = cond_soc

    cond_eco = res['SDG8'] | res['SDG9'] | res['SDG10'] | res['SDG12'] | res['SDG17']
    res.loc[:, 'Economy'] = cond_eco

    cond_env = res['SDG13'] | res['SDG14'] | res['SDG15']
    res.loc[:, 'Environment'] = cond_env

    cond_sdg = cond_env | cond_eco | cond_soc
    res.loc[:, 'SDG'] = cond_sdg

    cond_dt = res['AI'] | res['big_data'] | res['computing_infrastructure'] | res['IOT'] | res['robotics'] | res['blockchain'] | res['additive_manufacturing']
    res.loc[:, 'DT'] = cond_dt

    cond_inter = res['SDG'] & res['DT']
    res['Intersection'] = cond_inter

    dataframe_columns = [col for col in list(dataframe.columns) if col not in ['lst_DT', 'lst_SDG']]
    res_columns = dataframe_columns + ['SDG', 'DT', 'Intersection', 'Economy', 'Society', 'Environment']+ lst_sdg + lst_dt

    return res.loc[:, res_columns]

In [2]:
df = pd.read_pickle("../data/dataframes/full_pubs.pkl")

In [3]:
df

,PT,AU,AF,TI,SO,DT,DE,ID,AB,C1,...,SDG15,SDG16,SDG17,additive_manufacturing,AI,big_data,blockchain,computing_infrastructure,IOT,robotics
0,J,"Gusiatin, ZM; Kulikowska, D; Klik, B","Gusiatin, Zygmunt M.; Kulikowska, Dorota; Klik...",New-Generation Washing Agents in Remediation o...,INTERNATIONAL JOURNAL OF ENVIRONMENTAL RESEARC...,False,soil washing; soil flushing; heavy metals; soi...,DISSOLVED ORGANIC-MATTER; REMOVE HEAVY-METALS;...,Soil quality is seriously reduced due to chemi...,"[Gusiatin, Zygmunt M.; Kulikowska, Dorota; Kli...",...,False,False,False,False,False,False,False,False,False,False
1,J,"Vitko, TG; Suffet, IH","Vitko, Tadeo G.; Suffet, Irwin H. (Mel)",Corrective factors applied to reduced sulfur c...,WATER ENVIRONMENT RESEARCH,False,odor treatment design; odors; reduced sulfur o...,NaN,To determine accurately odorant concentrations...,"[Vitko, Tadeo G.] 26362 Via Conchita, Mission ...",...,False,False,False,False,False,False,False,False,False,False
2,C,"Ntasiou, M; Andreou, E","Ntasiou, M.; Andreou, E.",The Standard of Industrial Symbiosis. Environm...,SUSTAINABLE SYNERGIES FROM BUILDINGS TO THE UR...,False,Industrial; Business; Parks; Symbiosis; Standard,NaN,The siting of an Industrial park with environm...,"[Ntasiou, M.; Andreou, E.] Hellen Open Univ, S...",...,False,False,False,False,False,False,False,False,False,False
3,J,"Wu, LP; Zhang, SR; Chen, MM; Liu, J; Ding, XD","Wu, Lipeng; Zhang, Shirong; Chen, Mengmeng; Li...",A sustainable option: Biochar addition can imp...,ENVIRONMENTAL TECHNOLOGY & INNOVATION,False,Biochar; Phosphorus retention; Microbial bioma...,AGRICULTURAL SOILS; EXTRACTION METHOD; AVAILAB...,Application of straw and its derived biochar h...,"[Wu, Lipeng; Zhang, Shirong; Chen, Mengmeng; D...",...,False,False,False,False,False,False,False,False,False,False
4,C,"Staicu, D","Staicu, Daniela",Contributions of social entrepreneurship to te...,PROCEEDINGS OF THE INTERNATIONAL CONFERENCE ON...,False,textile waste prevention; clothing reuse; soci...,ENVIRONMENTAL-IMPACT; FASHION; REUSE,"The current system for producing, distributing...","[Staicu, Daniela] Bucharest Univ Econ Studies,...",...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2798083,C,"Dieffenderfer, J; Wilkins, M; Hood, C; Beppler...","Dieffenderfer, James; Wilkins, Michael; Hood, ...",Towards a Sweat-based Wireless and Wearable El...,2016 IEEE SENSORS,True,potentiostat; electrochemical; wearable sensin...,LACTATE CONCENTRATIONS; BIOSENSORS; BLOOD,The analysis of biomarkers within sweat is adv...,"[Dieffenderfer, James; Wilkins, Michael; Hood,...",...,False,False,False,False,False,False,False,False,True,False
2798084,C,"Dinesh, K; Xiong, ML; Adams, J; Dorsey, R; Sha...","Dinesh, Karthik; Xiong, Mulin; Adams, Jamie; D...",SIGNAL ANALYSIS FOR DETECTING MOTOR SYMPTOMS I...,2016 IEEE WESTERN NEW YORK IMAGE AND SIGNAL PR...,True,Wearable sensors; Parkinson's; Huntington's; a...,NaN,We report on a pilot study for signal processi...,"[Dinesh, Karthik; Sharma, Gaurav] Univ Rochest...",...,False,False,False,False,False,False,False,False,True,False
2798085,C,"Ding, CW; Heidari, S; Wang, YZ; Liu, YP; Hu, JT","Ding, Caiwen; Heidari, Soroush; Wang, Yanzhi; ...",Multi-Source In-Door Energy Harvesting for Non...,2016 IEEE INTERNATIONAL SYMPOSIUM ON CIRCUITS ...,True,NaN,CONVERTER,"Due to size, longevity, safety, and recharging...","[Ding, Caiwen; Wang, Yanzhi] Syracuse Univ, De...",...,False,False,False,False,False,False,False,False,True,False
2798086,C,"Ding, CW; Li, HJ; Hu, JT; Liu, YP; Wang, YZ","Ding, Caiwen; Li, Hongjia; Hu, Jingtong; Liu, ...",Dynamic Converter Reconfiguration for Near-Thr...,PROCEEDINGS OF THE 34TH IEEE INTERNATIONAL CON...,True,Non-volatile processor (NVP); Near-Threshold C...,NaN,Energy harvesting is becoming a preferred choi...,"[Ding, Caiwen; Li, Hongjia; Wang, Yanzhi] Syra...",...,False,False,False,False,False,False,False,

In [11]:
a = df[(df['SDG13']) & df['blockchain']]

In [12]:
b = iter(a['AB'])


In [26]:
c = next(b)
c

"Carbon emission is a prominent issue, and smart urban solutions have the technological capabilities to implement change. The technologies for creating smart energy systems already exist, some of which are currently under wide deployment globally. By investing in energy efficiency solutions (such as the smart meter), research shows that the end-user is able to not only save money, but also reduce their household's carbon footprint. Therefore, in this paper, the focus is on the end-user, and adopting a quantitative analysis of the perception of 1365 homes concerning the smart gas meter installation. The focus is on linking end-user attributes (age, education, social class and employment status) with their opinion on reducing energy, saving money, changing home behaviour and lowering carbon emissions. The results show that there is a statistical significance between certain attributes of end-users and their consideration of smart meters for making beneficial changes. In particular, the i

In [28]:
elem = a[a['AB'] == c]

In [29]:
elem.DE

13360    smart cities; smart meter; gas; climate change...
Name: DE, dtype: object

In [13]:
folder = "/media/kevin-desktop/My Passport/SDG/data/raw_data/raw_dt/AI/2022/"
import os
lst = [int(elem.split(".")[0])  for elem in os.listdir(folder) if "-" not in elem]
lst.sort()
for ind, elem in enumerate(lst, start=1):

    print(ind, elem)

1 1
2 2
3 3
4 4
5 5
6 6
7 7
8 8
9 9
10 10
11 11
12 12
13 13
14 14
15 15
16 16
17 17
18 18
19 19
20 20
21 21
22 22
23 23
24 24
25 25
26 26
27 27
28 28
29 29
30 30
31 31
32 32
33 33
34 34
35 35
36 36
37 37
38 38
39 39
40 40
41 41
42 42
43 43
44 44
45 45
46 46
47 47
48 48
49 49
50 50
51 51
52 52
53 53
54 54
55 55
56 56
57 57
58 58
59 59
60 60
61 61
62 62
63 63
64 64
65 65
66 66
67 67
68 68
69 69
70 70
71 71
72 72
73 73
74 74
75 75
76 76
77 77
78 78
79 79
80 80
81 81
82 82
83 83
84 84
85 85
86 86
87 87
88 88
89 89
90 90
91 91
92 92
93 93
94 94
95 95
96 96
97 97
98 98
99 99
100 100
101 101
102 102
103 103
104 104
105 105
106 106
107 107
108 108
109 109
110 110
111 111
112 112
113 113
114 114
115 115
116 116
117 117
118 118
119 119
120 120
121 121
122 122
123 123
124 124
125 125
126 126
127 127
128 128
129 129
130 130
131 131
132 132
133 133
134 134
135 135
136 136
137 137
138 138
139 139
140 140
141 141
142 142
143 143
144 144
145 145
146 146
147 147
148 148
149 149
150 150
151 151
152 152


In [1]:
import pandas as pd


In [14]:
df2022 = pd.read_parquet("../data/AI_2022.parquet")


In [24]:
path = "../data/dataframes/new_dt.pkl"
df = pd.read_pickle(path)

In [25]:
df

,PT,AU,AF,TI,SO,DT,DE,ID,AB,C1,...,Z9,PY,WC,DI,SC,UT,CN,lst_SDG,lst_DT,lst_Target
0,J,"Gillispie, C","Gillispie, Clara","How Can South Korea Teach, Lead, and Better En...",ASIA POLICY,Article,SOUTH KOREA; DATA GOVERNANCE; DATA PRIVACY; CO...,NaN,This essay explores how South Korea might cont...,"[Gillispie, Clara] Natl Bur Asian Res, Washing...",...,0.0,2021,Area Studies; International Relations,NaN,Area Studies; International Relations,WOS:000800262900014,United States,,computing_infrastructure,
1,J,"Paes, B; Li, A; Kim, D; Lanctot, KL; Mitchell, I","Paes, Bosco; Li, Abby; Kim, Doyoung; Lanctot, ...",A Comparison of Respiratory Syncytial Viral Pr...,AMERICAN JOURNAL OF PERINATOLOGY,Article,multiple births; singletons; respiratory syncy...,VIRUS MONOCLONAL-ANTIBODY; PRETERM INFANTS; IN...,Objective The aim of this study is to compare ...,"[Paes, Bosco] McMaster Univ, Dept Pediat, HSC-...",...,1.0,2021,Obstetrics & Gynecology; Pediatrics,10.1055/s-0040-1708804,Obstetrics & Gynecology; Pediatrics,WOS:000690384400018,"Canada, Canada, Canada, Canada, Canada",,computing_infrastructure,
2,J,"Chyzmar, I; Hoblyk, V","Chyzmar, Ivan; Hoblyk, Volodymyr",E-sports organizations with franchised network...,ECONOMIC ANNALS-XXI,Article,E-sport; Franchise; Blizzard Arena; Overwatch ...,NaN,The paper focuses on the priority given to the...,"[Chyzmar, Ivan] Mukachevo State Univ, Dept Eco...",...,0.0,2021,Economics,10.21003/ea.V187-15,Business & Economics,WOS:000716523300014,"Ukraine, Ukraine",,computing_infrastructure,
3,J,"Klein, I; Oppelt, N; Kuenzer, C","Klein, Igor; Oppelt, Natascha; Kuenzer, Claudia",Application of Remote Sensing Data for Locust ...,INSECTS,Review,locust monitoring; locust outbreak; remote sen...,ASIAN MIGRATORY LOCUST; HABITAT SUITABILITY IN...,Simple Summary Locust outbreaks around the wor...,"[Klein, Igor; Kuenzer, Claudia] German Aerosp ...",...,6.0,2021,Entomology,10.3390/insects12030233,Entomology,WOS:000633803300001,"Germany, Germany, Germany, Germany",,computing_infrastructure,
4,J,"Gong, XX; Zhang, B","Gong, Xinxin; Zhang, Bin",Resistance of SNOW-V against Fast Correlation ...,IACR TRANSACTIONS ON SYMMETRIC CRYPTOLOGY,Article,Stream ciphers; SNOW-V; FSM; Bitwise Fast corr...,STREAM CIPHERS; CRYPTANALYSIS; SOSEMANUK,SNOW-V is a new member in the SNOW family of s...,"[Gong, Xinxin; Zhang, Bin] State Key Lab Crypt...",...,7.0,2021,"Computer Science, Software Engineering; Comput...",10.46586/tosc.v2021.i1.378-410,Computer Science,WOS:000631653800012,"China, China, China",,computing_infrastructure,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2085373,C,"Dinesh, K; Xiong, ML; Adams, J; Dorsey, R; Sha...","Dinesh, Karthik; Xiong, Mulin; Adams, Jamie; D...",SIGNAL ANALYSIS FOR DETECTING MOTOR SYMPTOMS I...,2016 IEEE WESTERN NEW YORK IMAGE AND SIGNAL PR...,Proceedings Paper,Wearable sensors; Parkinson's; Huntington's; a...,NaN,We report on a pilot study for signal processi...,"[Dinesh, Karthik; Sharma, Gaurav] Univ Rochest...",...,1.0,2016,"Engineering, Electrical & Electronic",NaN,Engineering,WOS:000402630000005,"United States, United States, United States, U...",,IOT,
2085374,C,"Ding, CW; Heidari, S; Wang, YZ; Liu, YP; Hu, JT","Ding, Caiwen; Heidari, Soroush; Wang, Yanzhi; ...",Multi-Source In-Door Energy Harvesting for Non...,2016 IEEE INTERNATIONAL SYMPOSIUM ON CIRCUITS ...,Proceedings Paper,NaN,CONVERTER,"Due to size, longevity, safety, and recharging...","[Ding, Caiwen; Wang, Yanzhi] Syracuse Univ, De...",...,9.0,2016,"Engineering, Electrical & Electronic",NaN,Engineering,WOS:000390094700044,"United States, United States, United States, U...",,IOT,
2085375,C,"Ding, CW; Li, HJ; Hu, JT; Liu, YP; Wang, YZ","Ding, Caiwen; Li, Hongjia; Hu, Jingtong; Liu, ...",Dynamic Converter Reconfiguration for Near-Thr...,PROCEEDINGS OF THE 34TH IEEE INTERNATIONAL CON...,Proceedings Paper,Non-volatile processor (NVP); Near-Threshold C...,NaN,Energy harvesting is becoming a preferred choi...,"[Di

In [26]:
all = pd.concat([df2022, df])
all

,PT,AU,AF,TI,SO,DT,DE,ID,AB,C1,...,Z9,PY,WC,DI,SC,UT,CN,lst_DT,lst_SDG,lst_Target
0,C,"Kiseleva, EI; Astachova, IF","Kiseleva, E., I; Astachova, I. F.",Intelligent Support for Medical Decision Making,ADVANCES IN AUTOMATION III,Proceedings Paper,Diabetes mellitus; Artificial neural network; ...,NEURAL-NETWORK; EXPERT-SYSTEM; DIAGNOSIS; ALGO...,This paper presents the development and study ...,"[Kiseleva, E., I] Voronezh State Pedag Univ, 8...",...,1.0,2022,"Computer Science, Software Engineering; Comput...",10.1007/978-3-030-94202-1_11,Computer Science,WOS:000771714900011,"Russia, Russia",AI,NaN,NaN
1,J,"Abhishek, A; Anithaashri, TP","Abhishek, A.; Anithaashri, T. P.",YNOVEL WEB SEARCH FOR DATA ACCESSIBILITY USING...,INTERNATIONAL JOURNAL OF EARLY CHILDHOOD SPECI...,Article,Artificial Intelligence; Convolutional Neural ...,None,Aim: To improvise a web search for data access...,"[Abhishek, A.; Anithaashri, T. P.] Saveetha Un...",...,0.0,2022,"Education, Special",10.9756/INT-JECSE/V14I3.722,Education & Educational Research,WOS:000813443800047,"India, India",AI,NaN,NaN
2,C,"Rojek, I; Dostatni, E; Kotlarz, P","Rojek, Izabela; Dostatni, Ewa; Kotlarz, Piotr",Comparison of Neural Networks Aiding Material ...,INNOVATIONS IN MECHATRONICS ENGINEERING,Proceedings Paper,Compatibility; Neural networks; Classification...,PREDICTION; ECODESIGN; ENSEMBLE; DESIGN,A new method of selection of materials at the ...,"[Rojek, Izabela; Kotlarz, Piotr] Kazimierz Wie...",...,0.0,2022,"Engineering, Mechanical",10.1007/978-3-030-79168-1_2,Engineering,WOS:000773488800002,"Poland, Poland, Poland",AI,NaN,NaN
3,C,"Srun, N; Leang, S; Thu, YK; Sam, S","Srun, Nalin; Leang, Sotheara; Thu, Ye Kyaw; Sa...",Convolutional Time Delay Neural Network for Kh...,2022 17TH INTERNATIONAL JOINT SYMPOSIUM ON ART...,Proceedings Paper,Khmer ASR; Time Delay Neural Network; Convolut...,None,Convolutional Neural Networks have been proven...,"[Srun, Nalin; Thu, Ye Kyaw; Sam, Sethserey] Ca...",...,0.0,2022,None,10.1109/ISAI-NLP56921.2022.9960286,None,WOS:000900145700039,"Cambodia, Cambodia, Cambodia, France",AI,NaN,NaN
4,C,"Zhang, DF; Hou, TY; Yang, JK; Xiao, JJ","Zhang, Defu; Hou, Tongyu; Yang, Jiankun; Xiao,...",Study on Marine Diesel Engine Fault Identifica...,PROCEEDINGS OF 2022 IEEE INTERNATIONAL CONFERE...,Proceedings Paper,Marine diesel engine; Failure to identify; Rad...,EXPERT-SYSTEM; ALGORITHM,In order to further improve the accuracy and r...,"[Zhang, Defu; Hou, Tongyu] Tianjin Univ Techno...",...,0.0,2022,Automation & Control Systems; Mechanics,10.1109/ICMA54519.2022.9856137,Automation & Control Systems; Mechanics,WOS:000865966500249,"China, China, China, China",AI,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2085373,C,"Dinesh, K; Xiong, ML; Adams, J; Dorsey, R; Sha...","Dinesh, Karthik; Xiong, Mulin; Adams, Jamie; D...",SIGNAL ANALYSIS FOR DETECTING MOTOR SYMPTOMS I...,2016 IEEE WESTERN NEW YORK IMAGE AND SIGNAL PR...,Proceedings Paper,Wearable sensors; Parkinson's; Huntington's; a...,NaN,We report on a pilot study for signal processi...,"[Dinesh, Karthik; Sharma, Gaurav] Univ Rochest...",...,1.0,2016,"Engineering, Electrical & Electronic",NaN,Engineering,WOS:000402630000005,"United States, United States, United States, U...",IOT,,
2085374,C,"Ding, CW; Heidari, S; Wang, YZ; Liu, YP; Hu, JT","Ding, Caiwen; Heidari, Soroush; Wang, Yanzhi; ...",Multi-Source In-Door Energy Harvesting for Non...,2016 IEEE INTERNATIONAL SYMPOSIUM ON CIRCUITS ...,Proceedings Paper,NaN,CONVERTER,"Due to size, longevity, safety, and recharging...","[Ding, Caiwen; Wang, Yanzhi] Syracuse Univ, De...",...,9.0,2016,"Engineering, Electrical & Electronic",NaN,Engineering,WOS:000390094700044,"United States, United States, United States, U...",IOT,,
2085375,C,"Ding, CW; Li, HJ; Hu, JT; Liu, YP; Wang, YZ","Ding, Caiwen; Li, Hongjia; Hu, Jingtong; Liu, ...",Dynamic Converter Reconfiguration for Near-Thr...,PROCEEDINGS OF THE 34TH IEEE INTERNATIONAL CON...

In [27]:
all.drop_duplicates(subset='UT', inplace=True)
all

,PT,AU,AF,TI,SO,DT,DE,ID,AB,C1,...,Z9,PY,WC,DI,SC,UT,CN,lst_DT,lst_SDG,lst_Target
0,C,"Kiseleva, EI; Astachova, IF","Kiseleva, E., I; Astachova, I. F.",Intelligent Support for Medical Decision Making,ADVANCES IN AUTOMATION III,Proceedings Paper,Diabetes mellitus; Artificial neural network; ...,NEURAL-NETWORK; EXPERT-SYSTEM; DIAGNOSIS; ALGO...,This paper presents the development and study ...,"[Kiseleva, E., I] Voronezh State Pedag Univ, 8...",...,1.0,2022,"Computer Science, Software Engineering; Comput...",10.1007/978-3-030-94202-1_11,Computer Science,WOS:000771714900011,"Russia, Russia",AI,NaN,NaN
1,J,"Abhishek, A; Anithaashri, TP","Abhishek, A.; Anithaashri, T. P.",YNOVEL WEB SEARCH FOR DATA ACCESSIBILITY USING...,INTERNATIONAL JOURNAL OF EARLY CHILDHOOD SPECI...,Article,Artificial Intelligence; Convolutional Neural ...,None,Aim: To improvise a web search for data access...,"[Abhishek, A.; Anithaashri, T. P.] Saveetha Un...",...,0.0,2022,"Education, Special",10.9756/INT-JECSE/V14I3.722,Education & Educational Research,WOS:000813443800047,"India, India",AI,NaN,NaN
2,C,"Rojek, I; Dostatni, E; Kotlarz, P","Rojek, Izabela; Dostatni, Ewa; Kotlarz, Piotr",Comparison of Neural Networks Aiding Material ...,INNOVATIONS IN MECHATRONICS ENGINEERING,Proceedings Paper,Compatibility; Neural networks; Classification...,PREDICTION; ECODESIGN; ENSEMBLE; DESIGN,A new method of selection of materials at the ...,"[Rojek, Izabela; Kotlarz, Piotr] Kazimierz Wie...",...,0.0,2022,"Engineering, Mechanical",10.1007/978-3-030-79168-1_2,Engineering,WOS:000773488800002,"Poland, Poland, Poland",AI,NaN,NaN
3,C,"Srun, N; Leang, S; Thu, YK; Sam, S","Srun, Nalin; Leang, Sotheara; Thu, Ye Kyaw; Sa...",Convolutional Time Delay Neural Network for Kh...,2022 17TH INTERNATIONAL JOINT SYMPOSIUM ON ART...,Proceedings Paper,Khmer ASR; Time Delay Neural Network; Convolut...,None,Convolutional Neural Networks have been proven...,"[Srun, Nalin; Thu, Ye Kyaw; Sam, Sethserey] Ca...",...,0.0,2022,None,10.1109/ISAI-NLP56921.2022.9960286,None,WOS:000900145700039,"Cambodia, Cambodia, Cambodia, France",AI,NaN,NaN
4,C,"Zhang, DF; Hou, TY; Yang, JK; Xiao, JJ","Zhang, Defu; Hou, Tongyu; Yang, Jiankun; Xiao,...",Study on Marine Diesel Engine Fault Identifica...,PROCEEDINGS OF 2022 IEEE INTERNATIONAL CONFERE...,Proceedings Paper,Marine diesel engine; Failure to identify; Rad...,EXPERT-SYSTEM; ALGORITHM,In order to further improve the accuracy and r...,"[Zhang, Defu; Hou, Tongyu] Tianjin Univ Techno...",...,0.0,2022,Automation & Control Systems; Mechanics,10.1109/ICMA54519.2022.9856137,Automation & Control Systems; Mechanics,WOS:000865966500249,"China, China, China, China",AI,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2085372,C,"Dieffenderfer, J; Wilkins, M; Hood, C; Beppler...","Dieffenderfer, James; Wilkins, Michael; Hood, ...",Towards a Sweat-based Wireless and Wearable El...,2016 IEEE SENSORS,Proceedings Paper,potentiostat; electrochemical; wearable sensin...,LACTATE CONCENTRATIONS; BIOSENSORS; BLOOD,The analysis of biomarkers within sweat is adv...,"[Dieffenderfer, James; Wilkins, Michael; Hood,...",...,9.0,2016,"Engineering, Electrical & Electronic; Remote S...",NaN,Engineering; Remote Sensing,WOS:000399395700065,"United States, United States, United States, U...",IOT,,
2085373,C,"Dinesh, K; Xiong, ML; Adams, J; Dorsey, R; Sha...","Dinesh, Karthik; Xiong, Mulin; Adams, Jamie; D...",SIGNAL ANALYSIS FOR DETECTING MOTOR SYMPTOMS I...,2016 IEEE WESTERN NEW YORK IMAGE AND SIGNAL PR...,Proceedings Paper,Wearable sensors; Parkinson's; Huntington's; a...,NaN,We report on a pilot study for signal processi...,"[Dinesh, Karthik; Sharma, Gaurav] Univ Rochest...",...,1.0,2016,"Engineering, Electrical & Electronic",NaN,Engineering,WOS:000402630000005,"United States, United States, United States, U...",IOT,,
2085374,C,"Ding, CW; Heidari, S; Wang, YZ; Liu, YP; Hu, JT","Ding, Caiwen; Heidari, Soroush; Wang, Yanzhi; ...",Multi-Source In-Door

In [28]:
lenall = len(all)
lenall

1766601

In [29]:
lenPPA = len(all[all['DT'].isin(['Proceedings Paper', 'Article'])])
lenPPA

1673779

In [30]:
100 * (lenall-lenPPA) / lenall

5.254270771951335

In [31]:
all.PY.describe()

count    1.766601e+06
mean     2.017711e+03
std      3.280376e+00
min      2.010000e+03
25%      2.016000e+03
50%      2.018000e+03
75%      2.020000e+03
max      2.022000e+03
Name: PY, dtype: float64

In [1]:
import pandas as pd

In [2]:
path = "../data/dataframes/all_nodups.pkl"
df = pd.read_pickle(path)
df

,PT,AU,AF,TI,SO,DT,DE,ID,AB,C1,...,Z9,PY,WC,DI,SC,UT,CN,lst_DT,lst_Target,lst_SDG
0,J,"Gusiatin, ZM; Kulikowska, D; Klik, B","Gusiatin, Zygmunt M.; Kulikowska, Dorota; Klik...",New-Generation Washing Agents in Remediation o...,INTERNATIONAL JOURNAL OF ENVIRONMENTAL RESEARC...,Review,soil washing; soil flushing; heavy metals; soi...,DISSOLVED ORGANIC-MATTER; REMOVE HEAVY-METALS;...,Soil quality is seriously reduced due to chemi...,"[Gusiatin, Zygmunt M.; Kulikowska, Dorota; Kli...",...,16.0,2020,"Environmental Sciences; Public, Environmental ...",10.3390/ijerph17176220,"Environmental Sciences & Ecology; Public, Envi...",WOS:000570328900001,"Poland, Poland, Poland",{},{12.5},{SDG12}
1,J,"Vitko, TG; Suffet, IH","Vitko, Tadeo G.; Suffet, Irwin H. (Mel)",Corrective factors applied to reduced sulfur c...,WATER ENVIRONMENT RESEARCH,Article,odor treatment design; odors; reduced sulfur o...,NaN,To determine accurately odorant concentrations...,"[Vitko, Tadeo G.] 26362 Via Conchita, Mission ...",...,2.0,2021,"Engineering, Environmental; Environmental Scie...",10.1002/wer.1558,Engineering; Environmental Sciences & Ecology;...,WOS:000635505300001,"United States, United States",{},{12.5},{SDG12}
2,C,"Ntasiou, M; Andreou, E","Ntasiou, M.; Andreou, E.",The Standard of Industrial Symbiosis. Environm...,SUSTAINABLE SYNERGIES FROM BUILDINGS TO THE UR...,Proceedings Paper,Industrial; Business; Parks; Symbiosis; Standard,NaN,The siting of an Industrial park with environm...,"[Ntasiou, M.; Andreou, E.] Hellen Open Univ, S...",...,6.0,2017,Construction & Building Technology; Engineerin...,10.1016/j.proenv.2017.03.157,Construction & Building Technology; Engineerin...,WOS:000410925400094,"Greece, Greece",{},{12.5},{SDG12}
3,J,"Wu, LP; Zhang, SR; Chen, MM; Liu, J; Ding, XD","Wu, Lipeng; Zhang, Shirong; Chen, Mengmeng; Li...",A sustainable option: Biochar addition can imp...,ENVIRONMENTAL TECHNOLOGY & INNOVATION,Article,Biochar; Phosphorus retention; Microbial bioma...,AGRICULTURAL SOILS; EXTRACTION METHOD; AVAILAB...,Application of straw and its derived biochar h...,"[Wu, Lipeng; Zhang, Shirong; Chen, Mengmeng; D...",...,1.0,2021,Biotechnology & Applied Microbiology; Engineer...,10.1016/j.eti.2021.102070,Biotechnology & Applied Microbiology; Engineer...,WOS:000744257400019,"China, China, China, China, Canada",{},{12.5},{SDG12}
4,C,"Staicu, D","Staicu, Daniela",Contributions of social entrepreneurship to te...,PROCEEDINGS OF THE INTERNATIONAL CONFERENCE ON...,Proceedings Paper,textile waste prevention; clothing reuse; soci...,ENVIRONMENTAL-IMPACT; FASHION; REUSE,"The current system for producing, distributing...","[Staicu, Daniela] Bucharest Univ Econ Studies,...",...,1.0,2019,Business,10.2478/picbe-2019-0009,Business & Economics,WOS:000501603000009,Romania,{},"{12.5, 13.0}","{SDG13, SDG12}"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2798083,C,"Dieffenderfer, J; Wilkins, M; Hood, C; Beppler...","Dieffenderfer, James; Wilkins, Michael; Hood, ...",Towards a Sweat-based Wireless and Wearable El...,2016 IEEE SENSORS,Proceedings Paper,potentiostat; electrochemical; wearable sensin...,LACTATE CONCENTRATIONS; BIOSENSORS; BLOOD,The analysis of biomarkers within sweat is adv...,"[Dieffenderfer, James; Wilkins, Michael; Hood,...",...,9.0,2016,"Engineering, Electrical & Electronic; Remote S...",NaN,Engineering; Remote Sensing,WOS:000399395700065,"United States, United States, United States, U...",{IOT},{},{}
2798084,C,"Dinesh, K; Xiong, ML; Adams, J; Dorsey, R; Sha...","Dinesh, Karthik; Xiong, Mulin; Adams, Jamie; D...",SIGNAL ANALYSIS FOR DETECTING MOTOR SYMPTOMS I...,2016 IEEE WESTERN NEW YORK IMAGE AND SIGNAL PR...,Proceedings Paper,Wearable sensors; Parkinson's; Huntington's; a...,NaN,We report on a pilot study for signal processi...,"[Dinesh, Karthik; Sharma, Gaurav] Univ Rochest...",...,1.0,2016,"Engineering, Electrical & Electronic",NaN,Engineering,WOS:000402630000005,"United States, United States, United States

In [3]:
test = df.head(5)
test

,PT,AU,AF,TI,SO,DT,DE,ID,AB,C1,...,Z9,PY,WC,DI,SC,UT,CN,lst_DT,lst_Target,lst_SDG
0,J,"Gusiatin, ZM; Kulikowska, D; Klik, B","Gusiatin, Zygmunt M.; Kulikowska, Dorota; Klik...",New-Generation Washing Agents in Remediation o...,INTERNATIONAL JOURNAL OF ENVIRONMENTAL RESEARC...,Review,soil washing; soil flushing; heavy metals; soi...,DISSOLVED ORGANIC-MATTER; REMOVE HEAVY-METALS;...,Soil quality is seriously reduced due to chemi...,"[Gusiatin, Zygmunt M.; Kulikowska, Dorota; Kli...",...,16.0,2020,"Environmental Sciences; Public, Environmental ...",10.3390/ijerph17176220,"Environmental Sciences & Ecology; Public, Envi...",WOS:000570328900001,"Poland, Poland, Poland",{},{12.5},{SDG12}
1,J,"Vitko, TG; Suffet, IH","Vitko, Tadeo G.; Suffet, Irwin H. (Mel)",Corrective factors applied to reduced sulfur c...,WATER ENVIRONMENT RESEARCH,Article,odor treatment design; odors; reduced sulfur o...,NaN,To determine accurately odorant concentrations...,"[Vitko, Tadeo G.] 26362 Via Conchita, Mission ...",...,2.0,2021,"Engineering, Environmental; Environmental Scie...",10.1002/wer.1558,Engineering; Environmental Sciences & Ecology;...,WOS:000635505300001,"United States, United States",{},{12.5},{SDG12}
2,C,"Ntasiou, M; Andreou, E","Ntasiou, M.; Andreou, E.",The Standard of Industrial Symbiosis. Environm...,SUSTAINABLE SYNERGIES FROM BUILDINGS TO THE UR...,Proceedings Paper,Industrial; Business; Parks; Symbiosis; Standard,NaN,The siting of an Industrial park with environm...,"[Ntasiou, M.; Andreou, E.] Hellen Open Univ, S...",...,6.0,2017,Construction & Building Technology; Engineerin...,10.1016/j.proenv.2017.03.157,Construction & Building Technology; Engineerin...,WOS:000410925400094,"Greece, Greece",{},{12.5},{SDG12}
3,J,"Wu, LP; Zhang, SR; Chen, MM; Liu, J; Ding, XD","Wu, Lipeng; Zhang, Shirong; Chen, Mengmeng; Li...",A sustainable option: Biochar addition can imp...,ENVIRONMENTAL TECHNOLOGY & INNOVATION,Article,Biochar; Phosphorus retention; Microbial bioma...,AGRICULTURAL SOILS; EXTRACTION METHOD; AVAILAB...,Application of straw and its derived biochar h...,"[Wu, Lipeng; Zhang, Shirong; Chen, Mengmeng; D...",...,1.0,2021,Biotechnology & Applied Microbiology; Engineer...,10.1016/j.eti.2021.102070,Biotechnology & Applied Microbiology; Engineer...,WOS:000744257400019,"China, China, China, China, Canada",{},{12.5},{SDG12}
4,C,"Staicu, D","Staicu, Daniela",Contributions of social entrepreneurship to te...,PROCEEDINGS OF THE INTERNATIONAL CONFERENCE ON...,Proceedings Paper,textile waste prevention; clothing reuse; soci...,ENVIRONMENTAL-IMPACT; FASHION; REUSE,"The current system for producing, distributing...","[Staicu, Daniela] Bucharest Univ Econ Studies,...",...,1.0,2019,Business,10.2478/picbe-2019-0009,Business & Economics,WOS:000501603000009,Romania,{},"{12.5, 13.0}","{SDG13, SDG12}"


In [43]:
import copy
col = list(df.columns)
col[23] = 'DST'
col

['PT',
 'AU',
 'AF',
 'TI',
 'SO',
 'DT',
 'DE',
 'ID',
 'AB',
 'C1',
 'RP',
 'EM',
 'NR',
 'TC',
 'Z9',
 'PY',
 'WC',
 'DI',
 'SC',
 'UT',
 'CN',
 'lst_Target',
 'SDG',
 'DST',
 'Intersection',
 'Economy',
 'Society',
 'Environment',
 'SDG1',
 'SDG2',
 'SDG3',
 'SDG4',
 'SDG5',
 'SDG6',
 'SDG7',
 'SDG8',
 'SDG9',
 'SDG10',
 'SDG11',
 'SDG12',
 'SDG13',
 'SDG14',
 'SDG15',
 'SDG16',
 'SDG17',
 'additive_manufacturing',
 'AI',
 'big_data',
 'blockchain',
 'computing_infrastructure',
 'IOT',
 'robotics']

In [45]:
test.columns = col
test

,PT,AU,AF,TI,SO,DT,DE,ID,AB,C1,...,SDG15,SDG16,SDG17,additive_manufacturing,AI,big_data,blockchain,computing_infrastructure,IOT,robotics
0,J,"Gusiatin, ZM; Kulikowska, D; Klik, B","Gusiatin, Zygmunt M.; Kulikowska, Dorota; Klik...",New-Generation Washing Agents in Remediation o...,INTERNATIONAL JOURNAL OF ENVIRONMENTAL RESEARC...,False,soil washing; soil flushing; heavy metals; soi...,DISSOLVED ORGANIC-MATTER; REMOVE HEAVY-METALS;...,Soil quality is seriously reduced due to chemi...,"[Gusiatin, Zygmunt M.; Kulikowska, Dorota; Kli...",...,False,False,False,False,False,False,False,False,False,False
1,J,"Vitko, TG; Suffet, IH","Vitko, Tadeo G.; Suffet, Irwin H. (Mel)",Corrective factors applied to reduced sulfur c...,WATER ENVIRONMENT RESEARCH,False,odor treatment design; odors; reduced sulfur o...,NaN,To determine accurately odorant concentrations...,"[Vitko, Tadeo G.] 26362 Via Conchita, Mission ...",...,False,False,False,False,False,False,False,False,False,False
2,C,"Ntasiou, M; Andreou, E","Ntasiou, M.; Andreou, E.",The Standard of Industrial Symbiosis. Environm...,SUSTAINABLE SYNERGIES FROM BUILDINGS TO THE UR...,False,Industrial; Business; Parks; Symbiosis; Standard,NaN,The siting of an Industrial park with environm...,"[Ntasiou, M.; Andreou, E.] Hellen Open Univ, S...",...,False,False,False,False,False,False,False,False,False,False
3,J,"Wu, LP; Zhang, SR; Chen, MM; Liu, J; Ding, XD","Wu, Lipeng; Zhang, Shirong; Chen, Mengmeng; Li...",A sustainable option: Biochar addition can imp...,ENVIRONMENTAL TECHNOLOGY & INNOVATION,False,Biochar; Phosphorus retention; Microbial bioma...,AGRICULTURAL SOILS; EXTRACTION METHOD; AVAILAB...,Application of straw and its derived biochar h...,"[Wu, Lipeng; Zhang, Shirong; Chen, Mengmeng; D...",...,False,False,False,False,False,False,False,False,False,False
4,C,"Staicu, D","Staicu, Daniela",Contributions of social entrepreneurship to te...,PROCEEDINGS OF THE INTERNATIONAL CONFERENCE ON...,False,textile waste prevention; clothing reuse; soci...,ENVIRONMENTAL-IMPACT; FASHION; REUSE,"The current system for producing, distributing...","[Staicu, Daniela] Bucharest Univ Econ Studies,...",...,False,False,False,False,False,False,False,False,False,False


In [6]:
df
df.to_stata("../data/dataframes/giac/pubs.stata")

AttributeError: 'DataFrame' object has no attribute 'dtype'

In [4]:
path = "../data/dataframes/output/manual_annotation_df.pkl"
df = pd.read_pickle(path)

In [37]:
a = df.loc[df['Relevance'] == '5', ['TI', 'AB']]
a['tot'] = a['TI'] + "\n" + a['AB']

In [38]:
b = iter(a['tot'].to_list())

In [46]:
next(b)

'Can Cloud Computing Lead to Increased Sustainability of Mobile Devices?\nWhen it comes to looking at energy efficiency and the concept of sustainability in computing, the focus has invariably been on data centers and mobile infrastructures like cell towers, as these have been considered the power intensive within the computing sector. However, some recent studies show that energy consumed globally by mobile devices is increasingly a large fraction that must be improved. With increased emphasis on utilizing cloud computing through mobile devices, the study and analysis of the complete end-to-end scenario from mobile devices to end-servers is needed that includes a life cycle assessment of networked data centers that provide cloud services. This paper conducts a preliminary investigation into the energy benefits of offloading tasks from mobile end devices to powerful remote servers.'

In [1]:
import pandas as pd
df = pd.read_pickle("../data/dataframes/full_pubs.pkl")

In [2]:
a = df[df['TI'] == 'The future(s) of digital agriculture and sustainable food systems: An analysis of high-level policy documents']
a

,PT,AU,AF,TI,SO,DT,DE,ID,AB,C1,...,SDG15,SDG16,SDG17,additive_manufacturing,AI,big_data,blockchain,computing_infrastructure,IOT,robotics
617338,J,"Lajoie-O'Malley, A; Bronson, K; van der Burg, ...","Lajoie-O'Malley, Alana; Bronson, Kelly; van de...",The future(s) of digital agriculture and susta...,ECOSYSTEM SERVICES,Article,Digital transformation; Agriculture 4.0; Direc...,PRECISION AGRICULTURE; BIG DATA; EXPECTATIONS;...,Ecosystem services delivery is influenced by f...,"[Lajoie-O'Malley, Alana; Bronson, Kelly] Univ ...",...,False,False,False,False,False,True,False,False,False,False
